In [ ]:
import os
import pandas as pd

In [ ]:
IMG_DIR = "product_images"
os.makedirs(IMG_DIR, exist_ok=True)

df = pd.read_csv("preprocessedFoodFactsPhase2.csv")

In [ ]:
# --- product_id and image lookup ---

def extract_code_from_url(url):
    """Extract the product code string from /product/<code>/ in the URL."""
    url = str(url)
    if "/product/" in url:
        part = url.split("/product/")[1].split("/")[0]
        return part.strip()
    return None

def make_product_id(row):
    """
    Use the code from the URL as primary ID, because it matches filenames
    in product_images/. Fall back to a cleaned barcode or index.
    """
    # Prefer URL-based code
    code_from_url = extract_code_from_url(row.get("url", ""))
    if code_from_url:
        return code_from_url

    # Fallback: cleaned barcode (digits only, strip .0 etc.)
    bc = str(row.get("barcode", "")).strip()
    if bc and bc.lower() not in ["nan", "none"]:
        bc_digits = "".join(ch for ch in bc if ch.isdigit())
        if bc_digits:
            return bc_digits

    # Last fallback: index
    return f"idx_{row.name}"

df["product_id"] = df.apply(make_product_id, axis=1)